In [1]:
from sklearn.utils import resample
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
import os
import numpy as np
import pandas as pd

# テキストファイルをnumpy配列に読み込み
all_data = np.loadtxt("claim_toiawase_data_python.txt")
#numpy配列をDataFrame型に変換(でーたを見やすくするため)
all_data_pd = pd.DataFrame(all_data)
#DataFrame型の列名を設定
all_data_pd.columns = ["seq", "Imp", "Imp_val", "m_sum_neg", "c_neg", "m_sum_all", "m_sum_nzr", "Mag", "c_nzr", "c_all", "m_max", "s_min", "m_sdv_neg", 
                   "s_sdv_neg", "m_avg_neg", "s_avg_neg", "s_sdv_nzr", "m_sdv_all", "s_sdv_all", "s_avg_nzr", "m_avg_nzr", "s_avg_all", "Sc",
                   "m_avg_all", "m_sum_pos", "c_pos", "m_min", "s_max", "m_sdv_pos", "s_sdv_pos", "m_avg_pos", "s_avg_pos", "m_sdv_nzr"]
print(all_data_pd)   #ImpとImp_valは同じ

# 特徴量の部分のみall_Xに読み込み
all_X = all_data[:9813,3:33]
#ラベル部分をall_yに読み込み
all_y = all_data[:9813, 2]
print(all_X)
print(all_y)
# print(type(all_X))


         seq  Imp  Imp_val  m_sum_neg  c_neg  m_sum_all  m_sum_nzr  Mag  \
0        1.0  1.0      1.0        1.0    3.0        1.1        1.1  1.2   
1        2.0  1.0      1.0        0.5    1.0        0.5        0.5  0.5   
2        3.0  1.0      1.0        2.1    3.0        2.1        2.1  2.2   
3        4.0  1.0      1.0        1.9    3.0        1.9        1.9  2.0   
4        5.0  1.0      1.0        0.4    1.0        0.5        0.5  0.5   
5        6.0  1.0      1.0        0.5    1.0        0.5        0.5  0.5   
6        7.0  1.0      1.0        3.0    6.0        3.0        3.0  3.2   
7        8.0  1.0      1.0        1.7    3.0        2.6        2.6  2.8   
8        9.0  1.0      1.0        1.1    3.0        1.3        1.3  1.5   
9       10.0  1.0      1.0        1.3    2.0        1.3        1.3  1.3   
10      11.0  1.0      1.0        3.8    5.0        3.8        3.8  4.1   
11      12.0  1.0      1.0        1.0    2.0        1.0        1.0  1.0   
12      13.0  1.0      1.

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import EditedNearestNeighbours
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


for c in [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0]:
    for g in [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0]:
        
#       十回分のそれぞれのスコアを格納する空のlistを作成
        accuracy_train = []
        f1_train = []
        precision_train = []
        recall_train = []
        
        accuracy_test = []
        f1_test = []
        precision_test = []
        recall_test = []
#       パイプラインの作成
        pipe = make_pipeline(StandardScaler(), SVC(C=c, gamma=g, kernel="rbf", random_state=1))
        #層化10-fold交差検証の開始
        kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=2)
        for train_index, test_index in kf.split(all_X, all_y):
            train_X, test_X = all_X[train_index], all_X[test_index]
            train_y, test_y = all_y[train_index], all_y[test_index]
#             訓練データのラベル1のsmote前のデータ数の出力
#             print("train_X(label=1): %s" %train_X[train_y == 1].shape[0])
#             print("train_X(label=0): %s" %train_X[train_y == 0].shape[0])
            smote = SMOTE("minority", random_state=2)
#             train_X_smote , train_y_smote = smote.fit_sample(train_X, train_y)
#             print("smote_train_X(label=1): %s" %train_X_smote[ train_y_smote ==1 ].shape[0])
#             print("smote_train_X(label=0): %s" %train_X_smote[ train_y_smote ==0 ].shape[0])
            enn = EditedNearestNeighbours("all",random_state=2)
#             train_X_enn , train_y_enn = enn.fit_sample(train_X, train_y)
#             print("enn_train_X(label=1): %s" %train_X_enn[ train_y_enn ==1 ].shape[0])
#             print("enn_train_X(label=0): %s" %train_X_enn[ train_y_enn ==0 ].shape[0])
            #SMOTEENNを行い新しい訓練データを作成
            train_X_smoteenn , train_y_smoteenn = SMOTEENN(random_state=2, smote=smote,enn=enn ).fit_sample(train_X, train_y)
            print("smoteenn_train_X(label=1): %s" %train_X_smoteenn[ train_y_smoteenn ==1 ].shape[0])
            print("smoteenn_train_X(label=0): %s" %train_X_smoteenn[ train_y_smoteenn ==0 ].shape[0])
            pipe.fit(train_X_smoteenn, train_y_smoteenn)
            
            pre_label_train = pipe.predict(train_X_smoteenn)
            accuracy_train.append(accuracy_score(train_y_smoteenn, pre_label_train))
            f1_train.append(f1_score(train_y_smoteenn, pre_label_train))
            precision_train.append(precision_score(train_y_smoteenn, pre_label_train))
            recall_train.append(recall_score(train_y_smoteenn, pre_label_train))
            
            pre_label_test = pipe.predict(test_X)
            accuracy_test.append(accuracy_score(test_y, pre_label_test))
            f1_test.append(f1_score(test_y, pre_label_test))
            precision_test.append(precision_score(test_y, pre_label_test))
            recall_test.append(recall_score(test_y, pre_label_test))
        
        #十回のスコアの平均と標準偏差の出力
        print("\n")
        print("C:%s      gamma:%s" %(c,g))
        print("accuracy_train: %.4f +- %.4f" %(np.mean(accuracy_train), np.std(accuracy_train)))
        print("f1_train: %.4f +- %.4f" %(np.mean(f1_train), np.std(f1_train)))
        print("precision_train: %.4f +- %.4f" %(np.mean(precision_train), np.std(precision_train)))
        print("recall_train: %.4f +- %.4f" %(np.mean(recall_train), np.std(recall_train)))
        
        print("accuracy_test: %.4f +- %.4f" %(np.mean(accuracy_test), np.std(accuracy_test)))
        print("f1_test: %.4f +- %.4f" %(np.mean(f1_test), np.std(f1_test)))
        print("precision_test: %.4f +- %.4f" %(np.mean(precision_test), np.std(precision_test)))
        print("recall_test: %.4f +- %.4f" %(np.mean(recall_test), np.std(recall_test)))
        print("list_shape: %s" %len(accuracy_train))
        print("\n")
    

smoteenn_train_X(label=1): 5254
smoteenn_train_X(label=0): 5328


/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precisio

smoteenn_train_X(label=1): 5426
smoteenn_train_X(label=0): 5429


/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precisio

smoteenn_train_X(label=1): 5377
smoteenn_train_X(label=0): 5410


...

smoteenn_train_X(label=1): 5254
smoteenn_train_X(label=0): 5328
smoteenn_train_X(label=1): 5426
smoteenn_train_X(label=0): 5429
smoteenn_train_X(label=1): 5377
smoteenn_train_X(label=0): 5410
smoteenn_train_X(label=1): 5437
smoteenn_train_X(label=0): 5351
smoteenn_train_X(label=1): 5377
smoteenn_train_X(label=0): 5341
smoteenn_train_X(label=1): 5386
smoteenn_train_X(label=0): 5422
smoteenn_train_X(label=1): 5458
smoteenn_train_X(label=0): 5371
smoteenn_train_X(label=1): 5419
smoteenn_train_X(label=0): 5356
smoteenn_train_X(label=1): 5443
smoteenn_train_X(label=0): 5403
smoteenn_train_X(label=1): 5466
smoteenn_train_X(label=0): 5407


C:10.0      gamma:100.0
accuracy_train: 0.9856 +- 0.0031
f1_train: 0.9856 +- 0.0031
precision_train: 0.9912 +- 0.0037
recall_train: 0.9800 +- 0.0037 

accuracy_test: 0.6593 +- 0.0150
f1_test: 0.3918 +- 0.0109
precision_test: 0.2641 +- 0.0077
recall_test: 0.7606 +- 0.0415
list_shape: 10


smoteenn_train_X(label=1): 5254
smoteenn_train_X(label=0): 5328
smote

smoteenn_train_X(label=1): 5254
smoteenn_train_X(label=0): 5328
smoteenn_train_X(label=1): 5426
smoteenn_train_X(label=0): 5429
smoteenn_train_X(label=1): 5377
smoteenn_train_X(label=0): 5410
smoteenn_train_X(label=1): 5437
smoteenn_train_X(label=0): 5351
smoteenn_train_X(label=1): 5377
smoteenn_train_X(label=0): 5341
smoteenn_train_X(label=1): 5386
smoteenn_train_X(label=0): 5422
smoteenn_train_X(label=1): 5458
smoteenn_train_X(label=0): 5371
smoteenn_train_X(label=1): 5419
smoteenn_train_X(label=0): 5356
smoteenn_train_X(label=1): 5443
smoteenn_train_X(label=0): 5403
smoteenn_train_X(label=1): 5466
smoteenn_train_X(label=0): 5407


C:100.0      gamma:10.0
accuracy_train: 0.9792 +- 0.0027
f1_train: 0.9791 +- 0.0027
precision_train: 0.9865 +- 0.0045
recall_train: 0.9718 +- 0.0048 

accuracy_test: 0.7199 +- 0.0154
f1_test: 0.4266 +- 0.0184
precision_test: 0.3031 +- 0.0139
recall_test: 0.7224 +- 0.0451
list_shape: 10


smoteenn_train_X(label=1): 5254
smoteenn_train_X(label=0): 5328
smote

smoteenn_train_X(label=1): 5254
smoteenn_train_X(label=0): 5328
smoteenn_train_X(label=1): 5426
smoteenn_train_X(label=0): 5429
smoteenn_train_X(label=1): 5377
smoteenn_train_X(label=0): 5410
smoteenn_train_X(label=1): 5437
smoteenn_train_X(label=0): 5351
smoteenn_train_X(label=1): 5377
smoteenn_train_X(label=0): 5341
smoteenn_train_X(label=1): 5386
smoteenn_train_X(label=0): 5422
smoteenn_train_X(label=1): 5458
smoteenn_train_X(label=0): 5371
smoteenn_train_X(label=1): 5419
smoteenn_train_X(label=0): 5356
smoteenn_train_X(label=1): 5443
smoteenn_train_X(label=0): 5403
smoteenn_train_X(label=1): 5466
smoteenn_train_X(label=0): 5407


C:1000.0      gamma:1.0
accuracy_train: 0.9745 +- 0.0028
f1_train: 0.9742 +- 0.0029
precision_train: 0.9903 +- 0.0023
recall_train: 0.9586 +- 0.0053 

accuracy_test: 0.7952 +- 0.0160
f1_test: 0.4458 +- 0.0204
precision_test: 0.3675 +- 0.0228
recall_test: 0.5713 +- 0.0523
list_shape: 10


smoteenn_train_X(label=1): 5254
smoteenn_train_X(label=0): 5328
smote

smoteenn_train_X(label=1): 5254
smoteenn_train_X(label=0): 5328
smoteenn_train_X(label=1): 5426
smoteenn_train_X(label=0): 5429
smoteenn_train_X(label=1): 5377
smoteenn_train_X(label=0): 5410
smoteenn_train_X(label=1): 5437
smoteenn_train_X(label=0): 5351
smoteenn_train_X(label=1): 5377
smoteenn_train_X(label=0): 5341
smoteenn_train_X(label=1): 5386
smoteenn_train_X(label=0): 5422
smoteenn_train_X(label=1): 5458
smoteenn_train_X(label=0): 5371
smoteenn_train_X(label=1): 5419
smoteenn_train_X(label=0): 5356
smoteenn_train_X(label=1): 5443
smoteenn_train_X(label=0): 5403
smoteenn_train_X(label=1): 5466
smoteenn_train_X(label=0): 5407


C:10000.0      gamma:0.1
accuracy_train: 0.9688 +- 0.0031
f1_train: 0.9683 +- 0.0032
precision_train: 0.9860 +- 0.0043
recall_train: 0.9513 +- 0.0086 

accuracy_test: 0.7944 +- 0.0133
f1_test: 0.4548 +- 0.0209
precision_test: 0.3696 +- 0.0181
recall_test: 0.5960 +- 0.0594
list_shape: 10


smoteenn_train_X(label=1): 5254
smoteenn_train_X(label=0): 5328
smot

smoteenn_train_X(label=1): 5254
smoteenn_train_X(label=0): 5328
smoteenn_train_X(label=1): 5426
smoteenn_train_X(label=0): 5429
smoteenn_train_X(label=1): 5377
smoteenn_train_X(label=0): 5410
smoteenn_train_X(label=1): 5437
smoteenn_train_X(label=0): 5351
smoteenn_train_X(label=1): 5377
smoteenn_train_X(label=0): 5341
smoteenn_train_X(label=1): 5386
smoteenn_train_X(label=0): 5422
smoteenn_train_X(label=1): 5458
smoteenn_train_X(label=0): 5371
smoteenn_train_X(label=1): 5419
smoteenn_train_X(label=0): 5356
smoteenn_train_X(label=1): 5443
smoteenn_train_X(label=0): 5403
smoteenn_train_X(label=1): 5466
smoteenn_train_X(label=0): 5407


C:100000.0      gamma:0.01
accuracy_train: 0.9421 +- 0.0033
f1_train: 0.9411 +- 0.0034
precision_train: 0.9594 +- 0.0029
recall_train: 0.9236 +- 0.0076 

accuracy_test: 0.8014 +- 0.0101
f1_test: 0.4767 +- 0.0200
precision_test: 0.3851 +- 0.0161
recall_test: 0.6278 +- 0.0462
list_shape: 10


smoteenn_train_X(label=1): 5254
smoteenn_train_X(label=0): 5328
sm

In [12]:
a = np.array(accuracy_train)
print(np.mean(accuracy_train))
print(a.mean())


b = [1,1,1,1,1,1,1,0,0,0,0,0,1,0,1,0,1]
c = [0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0]
print(confusion_matrix(b,c,labels=[1,0]))
print(recall_score(b,c))
print(len(b))

print(all_X)
print(all_X[:, :9].shape)

1.0
1.0
[[2 8]
 [1 6]]
0.2
17
[[1.   3.   1.1  ... 0.1  0.1  0.34]
 [0.5  1.   0.5  ... 0.   0.   0.77]
 [2.1  3.   2.1  ... 0.   0.   0.91]
 ...
 [0.3  1.   0.3  ... 0.   0.   1.12]
 [0.1  1.   0.8  ... 0.35 0.35 0.68]
 [0.6  1.   0.6  ... 0.   0.   1.02]]
(9813, 9)
